<a href="https://colab.research.google.com/github/Mudit-Sharma24/IIT_Project/blob/main/ResearchAssistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai requests pathlib dataclasses argparse

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [ ]:
# Personal Research Assistant - Google Colab Version with Gemini API
# Fetches academic papers, summarizes key findings, and organizes notes using Google's Gemini API

# ========================================
# INSTALLATION AND SETUP
# ========================================

# Install required packages
!pip install google-generativeai requests xmltodict python-dotenv -q

# Import libraries
import os
import json
import requests
import time
from datetime import datetime
from pathlib import Path
from typing import List, Dict, Optional, Tuple
import xml.etree.ElementTree as ET
from urllib.parse import quote
import google.generativeai as genai
from dataclasses import dataclass
import logging
from google.colab import files
import zipfile
import shutil

# Configure logging for Colab
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# ========================================
# DATA CLASSES AND CONFIGURATIONS
# ========================================

@dataclass
class Paper:
    """Data class for academic papers"""
    title: str
    authors: List[str]
    abstract: str
    url: str
    published_date: str
    source: str
    doi: Optional[str] = None
    categories: List[str] = None

class ColabConfig:
    """Configuration class for Colab environment"""
    def __init__(self):
        self.base_path = Path("/content/research_projects")
        self.base_path.mkdir(exist_ok=True)

    def setup_gemini_key(self):
        """Interactive setup for Gemini API key"""
        from getpass import getpass

        print("Google Gemini API Key Setup")
        print("=" * 40)
        print("Get your FREE API key at: https://aistudio.google.com/app/apikey")
        print()

        # Check if key already exists in environment
        if os.getenv('GOOGLE_API_KEY'):
            print("Gemini API key already configured!")
            return os.getenv('GOOGLE_API_KEY')

        # Interactive key input
        api_key = getpass("Enter your Google Gemini API key (hidden input): ")

        if not api_key or len(api_key) < 20:
            raise ValueError("Invalid Gemini API key format. Please check your key.")

        # Set environment variable
        os.environ['GOOGLE_API_KEY'] = api_key
        genai.configure(api_key=api_key)
        print("API key configured successfully!")
        return api_key

# ========================================
# PAPER FETCHING CLASSES
# ========================================

class PaperFetcher:
    """Handles fetching papers from various academic sources"""

    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Research Assistant Bot 1.0 (Educational Purpose)'
        })

    def fetch_arxiv_papers(self, query: str, max_results: int = 10) -> List[Paper]:
        """Fetch papers from ArXiv"""
        print(f"Searching ArXiv for: '{query}'")

        base_url = "http://export.arxiv.org/api/query"
        params = {
            'search_query': f"all:{query}",
            'start': 0,
            'max_results': max_results,
            'sortBy': 'submittedDate',
            'sortOrder': 'descending'
        }

        try:
            response = self.session.get(base_url, params=params, timeout=30)
            response.raise_for_status()

            papers = []
            root = ET.fromstring(response.content)

            for entry in root.findall('{http://www.w3.org/2005/Atom}entry'):
                title = entry.find('{http://www.w3.org/2005/Atom}title').text.strip()
                abstract = entry.find('{http://www.w3.org/2005/Atom}summary').text.strip()

                authors = []
                for author in entry.findall('{http://www.w3.org/2005/Atom}author'):
                    name = author.find('{http://www.w3.org/2005/Atom}name').text
                    authors.append(name)

                published = entry.find('{http://www.w3.org/2005/Atom}published').text
                url = entry.find('{http://www.w3.org/2005/Atom}id').text

                # Extract categories
                categories = []
                for category in entry.findall('{http://www.w3.org/2005/Atom}category'):
                    categories.append(category.get('term'))

                paper = Paper(
                    title=title,
                    authors=authors,
                    abstract=abstract,
                    url=url,
                    published_date=published,
                    source="ArXiv",
                    categories=categories
                )
                papers.append(paper)

            print(f"Found {len(papers)} papers from ArXiv")
            return papers

        except Exception as e:
            print(f"Error fetching ArXiv papers: {e}")
            return []

    def fetch_semantic_scholar_papers(self, query: str, max_results: int = 10) -> List[Paper]:
        """Fetch papers from Semantic Scholar"""
        print(f"Searching Semantic Scholar for: '{query}'")

        base_url = "https://api.semanticscholar.org/graph/v1/paper/search"
        params = {
            'query': query,
            'limit': max_results,
            'fields': 'title,authors,abstract,url,publicationDate,externalIds'
        }

        try:
            response = self.session.get(base_url, params=params, timeout=30)
            response.raise_for_status()
            data = response.json()

            papers = []
            for item in data.get('data', []):
                if not item.get('abstract'):
                    continue

                authors = [author.get('name', '') for author in item.get('authors', [])]
                external_ids = item.get('externalIds', {})
                doi = external_ids.get('DOI')

                paper = Paper(
                    title=item.get('title', ''),
                    authors=authors,
                    abstract=item.get('abstract', ''),
                    url=item.get('url', ''),
                    published_date=item.get('publicationDate', ''),
                    source="Semantic Scholar",
                    doi=doi
                )
                papers.append(paper)

            print(f"Found {len(papers)} papers from Semantic Scholar")
            return papers

        except Exception as e:
            print(f"Error fetching Semantic Scholar papers: {e}")
            return []

    def fetch_pubmed_papers(self, query: str, max_results: int = 10) -> List[Paper]:
        """Fetch papers from PubMed (bonus source)"""
        print(f"Searching PubMed for: '{query}'")

        # First, search for paper IDs
        search_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
        search_params = {
            'db': 'pubmed',
            'term': query,
            'retmax': max_results,
            'retmode': 'json',
            'sort': 'pub_date'
        }

        try:
            search_response = self.session.get(search_url, params=search_params, timeout=30)
            search_response.raise_for_status()
            search_data = search_response.json()

            paper_ids = search_data.get('esearchresult', {}).get('idlist', [])

            if not paper_ids:
                print("No papers found from PubMed")
                return []

            # Fetch paper details
            fetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
            fetch_params = {
                'db': 'pubmed',
                'id': ','.join(paper_ids),
                'retmode': 'xml'
            }

            fetch_response = self.session.get(fetch_url, params=fetch_params, timeout=30)
            fetch_response.raise_for_status()

            papers = []
            root = ET.fromstring(fetch_response.content)

            for article in root.findall('.//PubmedArticle'):
                try:
                    # Extract title
                    title_elem = article.find('.//ArticleTitle')
                    title = title_elem.text if title_elem is not None else "No title"

                    # Extract abstract
                    abstract_elem = article.find('.//AbstractText')
                    abstract = abstract_elem.text if abstract_elem is not None else "No abstract available"

                    # Extract authors
                    authors = []
                    for author in article.findall('.//Author'):
                        lastname = author.find('LastName')
                        firstname = author.find('ForeName')
                        if lastname is not None and firstname is not None:
                            authors.append(f"{firstname.text} {lastname.text}")

                    # Extract publication date
                    pub_date = article.find('.//PubDate/Year')
                    published_date = pub_date.text if pub_date is not None else "Unknown"

                    # Extract PMID for URL
                    pmid_elem = article.find('.//PMID')
                    pmid = pmid_elem.text if pmid_elem is not None else ""
                    url = f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/" if pmid else ""

                    paper = Paper(
                        title=title,
                        authors=authors,
                        abstract=abstract,
                        url=url,
                        published_date=published_date,
                        source="PubMed"
                    )
                    papers.append(paper)

                except Exception as e:
                    continue  # Skip problematic papers

            print(f"Found {len(papers)} papers from PubMed")
            return papers

        except Exception as e:
            print(f"Error fetching PubMed papers: {e}")
            return []

# ========================================
# SUMMARIZATION CLASS WITH GEMINI
# ========================================

class GeminiResearchSummarizer:
    """Handles summarization using Google's Gemini API"""

    def __init__(self, api_key: str, model_name: str = "gemini-1.5-flash"):
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel(model_name)
        self.model_name = model_name

    def summarize_paper(self, paper: Paper) -> Dict[str, str]:
        """Generate a comprehensive summary of a paper"""
        print(f"Summarizing: {paper.title[:60]}...")

        prompt = f"""
        Please analyze this academic paper and provide a structured summary in JSON format:

        Title: {paper.title}
        Authors: {', '.join(paper.authors)}
        Abstract: {paper.abstract}
        Source: {paper.source}

        Provide your analysis in the following JSON format:
        {{
            "key_findings": "Main discoveries and results (2-3 sentences)",
            "methodology": "Research methods used (1-2 sentences)",
            "significance": "Why this research matters (1-2 sentences)",
            "limitations": "Study limitations or future work needed (1-2 sentences)",
            "keywords": ["keyword1", "keyword2", "keyword3", "keyword4", "keyword5"],
            "summary": "Comprehensive overview (4-5 sentences)",
            "practical_applications": "Real-world applications (1-2 sentences)"
        }}

        Be concise but informative. Focus on the most important aspects. Return only valid JSON.
        """

        try:
            response = self.model.generate_content(prompt)
            content = response.text.strip()

            # Try to parse JSON response
            try:
                # Clean the response to extract JSON
                if '```json' in content:
                    content = content.split('```json')[1].split('```')[0]
                elif '```' in content:
                    content = content.split('```')[1].split('```')[0]

                return json.loads(content)
            except json.JSONDecodeError:
                # Fallback summary
                return {
                    "key_findings": "Could not parse structured analysis",
                    "methodology": "Not specified",
                    "significance": "Not specified",
                    "limitations": "Not specified",
                    "keywords": ["analysis", "research", "academic"],
                    "summary": content[:500] + "..." if len(content) > 500 else content,
                    "practical_applications": "Not specified"
                }

        except Exception as e:
            print(f"Error summarizing paper: {e}")
            return {
                "key_findings": "Summary generation failed",
                "methodology": "Not available",
                "significance": "Not available",
                "limitations": "Not available",
                "keywords": ["error"],
                "summary": paper.abstract[:500] + "..." if len(paper.abstract) > 500 else paper.abstract,
                "practical_applications": "Not available"
            }

    def generate_literature_review(self, papers: List[Paper], summaries: List[Dict], query: str) -> str:
        """Generate a comprehensive literature review"""
        print("Generating literature review...")

        paper_summaries = []
        for paper, summary in zip(papers, summaries):
            paper_summaries.append({
                "title": paper.title,
                "authors": paper.authors[:3],  # Limit authors for brevity
                "key_findings": summary.get("key_findings", ""),
                "significance": summary.get("significance", ""),
                "keywords": summary.get("keywords", [])[:5]
            })

        prompt = f"""
        Write a comprehensive literature review based on the following research papers about "{query}":

        Number of papers analyzed: {len(papers)}

        Paper summaries:
        {json.dumps(paper_summaries, indent=2)}

        Your literature review should include:

        1. **Introduction** - Overview of the research area and its importance
        2. **Current State of Research** - Key themes and findings across studies
        3. **Methodological Approaches** - Common research methods used
        4. **Key Findings and Trends** - Major discoveries and patterns
        5. **Gaps and Limitations** - Areas needing further research
        6. **Future Directions** - Suggestions for upcoming research
        7. **Conclusion** - Summary of the field's current status

        Write in an academic style, approximately 1000-1200 words. Use clear headings and make connections between different studies.
        Cite papers by their titles when referencing specific findings.
        """

        try:
            response = self.model.generate_content(prompt)
            return response.text.strip()

        except Exception as e:
            print(f"Error generating literature review: {e}")
            return f"# Literature Review: {query}\n\nLiterature review generation encountered an error. Please review individual paper summaries for insights."

    def generate_research_insights(self, papers: List[Paper], summaries: List[Dict], query: str) -> str:
        """Generate additional research insights and trends"""
        print("Generating research insights...")

        keywords_all = []
        for summary in summaries:
            keywords_all.extend(summary.get("keywords", []))

        # Count keyword frequency
        keyword_counts = {}
        for keyword in keywords_all:
            keyword_counts[keyword] = keyword_counts.get(keyword, 0) + 1

        top_keywords = sorted(keyword_counts.items(), key=lambda x: x[1], reverse=True)[:10]

        prompt = f"""
        Based on the research about "{query}", provide insights about trends and patterns:

        Total papers analyzed: {len(papers)}
        Top keywords found: {[kw[0] for kw in top_keywords[:5]]}

        Generate insights covering:

        1. **Emerging Trends** - What new directions are researchers exploring?
        2. **Research Gaps** - What important questions remain unanswered?
        3. **Methodological Evolution** - How are research methods changing?
        4. **Cross-disciplinary Connections** - How does this field connect to others?
        5. **Future Predictions** - Where might this field be heading?

        Keep each section to 2-3 paragraphs. Be analytical and forward-thinking.
        """

        try:
            response = self.model.generate_content(prompt)
            return response.text.strip()

        except Exception as e:
            print(f" Error generating insights: {e}")
            return "Research insights generation failed."

# ========================================
# FILE ORGANIZATION CLASS
# ========================================

class ColabResearchOrganizer:
    """Handles file organization optimized for Colab"""

    def __init__(self, base_path: str = "/content/research_projects"):
        self.base_path = Path(base_path)
        self.base_path.mkdir(exist_ok=True)

    def create_project_folder(self, project_name: str) -> Path:
        """Create a new research project folder"""
        project_path = self.base_path / self.sanitize_filename(project_name)
        project_path.mkdir(exist_ok=True)

        # Create subfolders
        (project_path / "papers").mkdir(exist_ok=True)
        (project_path / "summaries").mkdir(exist_ok=True)
        (project_path / "notes").mkdir(exist_ok=True)
        (project_path / "data").mkdir(exist_ok=True)
        (project_path / "insights").mkdir(exist_ok=True)

        return project_path

    def sanitize_filename(self, filename: str) -> str:
        """Sanitize filename for cross-platform compatibility"""
        invalid_chars = '<>:"/\\|?*'
        for char in invalid_chars:
            filename = filename.replace(char, '_')
        return filename[:50]

    def save_paper_data(self, project_path: Path, papers: List[Paper]):
        """Save raw paper data"""
        papers_data = []
        for paper in papers:
            papers_data.append({
                "title": paper.title,
                "authors": paper.authors,
                "abstract": paper.abstract,
                "url": paper.url,
                "published_date": paper.published_date,
                "source": paper.source,
                "doi": paper.doi,
                "categories": paper.categories
            })

        with open(project_path / "data" / "papers_data.json", 'w', encoding='utf-8') as f:
            json.dump(papers_data, f, indent=2, ensure_ascii=False)

        print(f"Saved {len(papers)} papers to papers_data.json")

    def save_summaries(self, project_path: Path, papers: List[Paper], summaries: List[Dict]):
        """Save paper summaries"""
        all_summaries = []

        for paper, summary in zip(papers, summaries):
            filename = self.sanitize_filename(paper.title) + ".json"

            summary_data = {
                "title": paper.title,
                "authors": paper.authors,
                "url": paper.url,
                "source": paper.source,
                "published_date": paper.published_date,
                "summary": summary,
                "generated_on": datetime.now().isoformat()
            }

            # Save individual summary
            with open(project_path / "summaries" / filename, 'w', encoding='utf-8') as f:
                json.dump(summary_data, f, indent=2, ensure_ascii=False)

            all_summaries.append(summary_data)

        # Save consolidated summaries
        with open(project_path / "data" / "all_summaries.json", 'w', encoding='utf-8') as f:
            json.dump(all_summaries, f, indent=2, ensure_ascii=False)

        print(f"Saved {len(summaries)} summaries")

    def save_literature_review(self, project_path: Path, review: str, query: str):
        """Save literature review"""
        review_content = f"""# Literature Review: {query}

**Generated on:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**Research Query:** {query}
**AI Model:** Google Gemini

---

{review}

---

*This literature review was automatically generated by the Personal Research Assistant using Google's Gemini AI.*
"""

        with open(project_path / "notes" / "literature_review.md", 'w', encoding='utf-8') as f:
            f.write(review_content)

        print("Literature review saved")

    def save_research_insights(self, project_path: Path, insights: str, query: str):
        """Save research insights"""
        insights_content = f"""# Research Insights: {query}

**Generated on:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**Research Query:** {query}
**AI Model:** Google Gemini

---

{insights}

---

*These insights were automatically generated by the Personal Research Assistant using Google's Gemini AI.*
"""

        with open(project_path / "insights" / "research_insights.md", 'w', encoding='utf-8') as f:
            f.write(insights_content)

        print("Research insights saved")

    def create_project_summary(self, project_path: Path, query: str, papers: List[Paper]):
        """Create a comprehensive project summary"""
        sources = {}
        for paper in papers:
            sources[paper.source] = sources.get(paper.source, 0) + 1

        summary_content = f"""# Research Project: {query}

**Created:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**Query:** "{query}"
**Papers Found:** {len(papers)}
**AI Model:** Google Gemini (Free API)

##  Paper Sources
"""

        for source, count in sources.items():
            summary_content += f"- **{source}:** {count} papers\n"

        summary_content += f"""
## Project Structure

```
{project_path.name}/
├──  README.md (this file)
├── data/
│   ├── papers_data.json      # Raw paper metadata
│   └── all_summaries.json    # Consolidated summaries
├── summaries/                # Individual paper analyses
├── notes/
│   └── literature_review.md  # Generated literature review
├── insights/
│   └── research_insights.md  # Research trends and patterns
└── papers/                   # Additional paper files
```

## Papers Analyzed

"""

        for i, paper in enumerate(papers, 1):
            authors_str = ', '.join(paper.authors[:3])
            if len(paper.authors) > 3:
                authors_str += f" et al. ({len(paper.authors)} total)"

            summary_content += f"""### {i}. {paper.title}

- **Authors:** {authors_str}
- **Source:** {paper.source}
- **Published:** {paper.published_date}
- **URL:** [{paper.url}]({paper.url})

"""

        summary_content += f"""
## Next Steps

1. **Review Literature Review** - Check `notes/literature_review.md` for comprehensive analysis
2. **Explore Research Insights** - Browse `insights/research_insights.md` for trends and patterns
3. **Examine Individual Summaries** - Browse the `summaries/` folder for detailed paper analyses
4. **Download Results** - Use the download function to save your research locally
5. **Extend Research** - Add more papers or create additional analyses

##  How to Download

Run the following code to download your complete research project:

```python
research_assistant.download_project("{project_path.name}")
```

## Powered By

- **AI Model:** Google Gemini (Free API)
- **Paper Sources:** ArXiv, Semantic Scholar, PubMed
- **Environment:** Google Colab

---
*Generated by Personal Research Assistant for Google Colab*
"""

        with open(project_path / "README.md", 'w', encoding='utf-8') as f:
            f.write(summary_content)

        print("Project summary created")

    def create_downloadable_zip(self, project_path: Path) -> str:
        """Create a downloadable zip file of the project"""
        zip_path = f"/content/{project_path.name}.zip"

        with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for file_path in project_path.rglob('*'):
                if file_path.is_file():
                    arcname = file_path.relative_to(project_path.parent)
                    zipf.write(file_path, arcname)

        return zip_path

# ========================================
# MAIN RESEARCH ASSISTANT CLASS
# ========================================

class ColabResearchAssistant:
    """Main research assistant optimized for Google Colab with Gemini API"""

    def __init__(self, gemini_api_key: str, model_name: str = "gemini-1.5-flash"):
        self.fetcher = PaperFetcher()
        self.summarizer = GeminiResearchSummarizer(gemini_api_key, model_name)
        self.organizer = ColabResearchOrganizer()
        self.current_project_path = None

    def conduct_research(self, query: str, max_papers_per_source: int = 8,
                        project_name: Optional[str] = None,
                        sources: List[str] = ["arxiv", "semantic_scholar", "pubmed"]) -> Path:
        """Conduct complete research process with progress indicators"""

        if not project_name:
            project_name = f"{query.replace(' ', '_')[:30]}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

        print("STARTING RESEARCH PROJECT")
        print("=" * 50)
        print(f"Project: {project_name}")
        print(f" Query: '{query}'")
        print(f"Max papers per source: {max_papers_per_source}")
        print(f"Sources: {', '.join(sources)}")
        print(f"AI Model: Google Gemini")
        print()

        # Create project folder
        project_path = self.organizer.create_project_folder(project_name)
        self.current_project_path = project_path
        print(f"Project folder created: {project_path}")
        print()

        # Fetch papers from selected sources
        all_papers = []

        if "arxiv" in sources:
            arxiv_papers = self.fetcher.fetch_arxiv_papers(query, max_papers_per_source)
            all_papers.extend(arxiv_papers)

        if "semantic_scholar" in sources:
            ss_papers = self.fetcher.fetch_semantic_scholar_papers(query, max_papers_per_source)
            all_papers.extend(ss_papers)

        if "pubmed" in sources:
            pubmed_papers = self.fetcher.fetch_pubmed_papers(query, max_papers_per_source)
            all_papers.extend(pubmed_papers)

        if not all_papers:
            print("No papers found for the given query")
            return project_path

        print(f"\n PAPER COLLECTION SUMMARY")
        print("-" * 30)
        print(f"Total papers found: {len(all_papers)}")

        # Remove duplicates
        unique_papers = self.remove_duplicate_papers(all_papers)
        print(f"Unique papers after deduplication: {len(unique_papers)}")
        print()

        # Save paper data
        self.organizer.save_paper_data(project_path, unique_papers)

        # Generate summaries with progress tracking
        print("GENERATING AI SUMMARIES WITH GEMINI")
        print("-" * 40)
        summaries = []
        for i, paper in enumerate(unique_papers, 1):
            print(f" Processing {i}/{len(unique_papers)}: {paper.title[:50]}...")
            summary = self.summarizer.summarize_paper(paper)
            summaries.append(summary)

            # Rate limiting for Gemini API (more generous than OpenAI)
            if i < len(unique_papers):
                time.sleep(1)  # Reduced delay

        print(f"Generated {len(summaries)} paper summaries")
        print()

        # Save summaries
        self.organizer.save_summaries(project_path, unique_papers, summaries)

        # Generate literature review
        print("GENERATING LITERATURE REVIEW")
        print("-" * 30)
        literature_review = self.summarizer.generate_literature_review(unique_papers, summaries, query)
        self.organizer.save_literature_review(project_path, literature_review, query)
        print(" Literature review generated")
        print()

        # Generate research insights
        print("GENERATING RESEARCH INSIGHTS")
        print("-" * 30)
        insights = self.summarizer.generate_research_insights(unique_papers, summaries, query)
        self.organizer.save_research_insights(project_path, insights, query)
        print("Research insights generated")
        print()

        # Create project summary
        self.organizer.create_project_summary(project_path, query, unique_papers)

        print("RESEARCH PROJECT COMPLETED!")
        print("=" * 50)
        print(f"Location: {project_path}")
        print(f"Papers analyzed: {len(unique_papers)}")
        print(f"Files generated: {len(list(project_path.rglob('*')))} files")
        print(f"Powered by: Google Gemini API")
        print()
        print("To explore your results:")
        print(f"   • Project overview: {project_path}/README.md")
        print(f"   • Literature review: {project_path}/notes/literature_review.md")
        print(f"   • Research insights: {project_path}/insights/research_insights.md")
        print(f"   • Paper summaries: {project_path}/summaries/")
        print()

        return project_path

    def remove_duplicate_papers(self, papers: List[Paper]) -> List[Paper]:
        """Remove duplicate papers based on title similarity"""
        unique_papers = []
        seen_titles = set()

        for paper in papers:
            normalized_title = paper.title.lower().strip().replace(' ', '')
            if normalized_title not in seen_titles:
                seen_titles.add(normalized_title)
                unique_papers.append(paper)

        return unique_papers

    def display_project_summary(self, project_path: Path = None):
        """Display a nice summary of the research project"""
        if project_path is None:
            project_path = self.current_project_path

        if not project_path or not project_path.exists():
            print("No project found to display")
            return

        # Read project data
        papers_file = project_path / "data" / "papers_data.json"
        if papers_file.exists():
            with open(papers_file, 'r') as f:
                papers_data = json.load(f)

            print("PROJECT SUMMARY")
            print("=" * 40)
            print(f" Project: {project_path.name}")
            print(f" Papers: {len(papers_data)}")

            # Source breakdown
            sources = {}
            for paper in papers_data:
                source = paper['source']
                sources[source] = sources.get(source, 0) + 1

            print("\n Sources:")
            for source, count in sources.items():
                print(f"   • {source}: {count} papers")

            print(f"\n Files created: {len(list(project_path.rglob('*')))} files")
            print(f" Total size: {self.get_folder_size(project_path):.2f} MB")
            print(f" AI Model: Google Gemini")

    def get_folder_size(self, folder_path: Path) -> float:
        """Calculate folder size in MB"""
        total_size = 0
        for file_path in folder_path.rglob('*'):
            if file_path.is_file():
                total_size += file_path.stat().st_size
        return total_size / (1024 * 1024)

    def download_project(self, project_name: str = None):
        """Create and download a zip file of the research project"""
        if project_name:
            project_path = self.organizer.base_path / project_name
        else:
            project_path = self.current_project_path

        if not project_path or not project_path.exists():
            print("Project not found")
            return

        print(f"Creating download package for: {project_path.name}")
        zip_path = self.organizer.create_downloadable_zip(project_path)

        print(f"Download package ready!")
        print(f" Size: {Path(zip_path).stat().st_size / (1024*1024):.2f} MB")

        # Download the file
        files.download(zip_path)
        print(" Download started! Check your browser's downloads folder.")

    def quick_search(self, query: str, source: str = "arxiv", max_results: int = 5) -> List[Dict]:
        """Quick search function for immediate results without full processing"""
        print(f"Quick search: '{query}' from {source}")

        papers = []
        if source.lower() == "arxiv":
            papers = self.fetcher.fetch_arxiv_papers(query, max_results)
        elif source.lower() == "semantic_scholar":
            papers = self.fetcher.fetch_semantic_scholar_papers(query, max_results)
        elif source.lower() == "pubmed":
            papers = self.fetcher.fetch_pubmed_papers(query, max_results)

        # Return simplified data
        results = []
        for paper in papers:
            results.append({
                "title": paper.title,
                "authors": ", ".join(paper.authors[:3]),
                "source": paper.source,
                "url": paper.url,
                "abstract": paper.abstract[:200] + "..." if len(paper.abstract) > 200 else paper.abstract
            })

        return results

    def list_projects(self):
        """List all existing research projects"""
        projects = [d for d in self.organizer.base_path.iterdir() if d.is_dir()]

        if not projects:
            print("No research projects found.")
            return

        print("EXISTING RESEARCH PROJECTS")
        print("=" * 40)

        for i, project in enumerate(sorted(projects), 1):
            # Try to read project info
            readme_path = project / "README.md"
            papers_path = project / "data" / "papers_data.json"

            paper_count = 0
            if papers_path.exists():
                try:
                    with open(papers_path, 'r') as f:
                        paper_count = len(json.load(f))
                except:
                    pass

            size_mb = self.get_folder_size(project)
            print(f"{i}. {project.name}")
            print(f"   Papers: {paper_count} | 💾 Size: {size_mb:.1f}MB")
            print(f"   Path: {project}")
            print()

# ========================================
# EASY-TO-USE FUNCTIONS FOR COLAB
# ========================================

def quick_research(query: str, max_papers: int = 10, model: str = "gemini-1.5-flash"):
    """
    Quick research function - just provide a query!

    Args:
        query (str): Your research question or topic
        max_papers (int): Maximum papers per source (default: 10)
        model (str): Gemini model to use (default: gemini-1.5-flash for speed)

    Returns:
        ColabResearchAssistant: The research assistant instance
    """

    # Setup configuration
    config = ColabConfig()
    api_key = config.setup_gemini_key()

    # Initialize research assistant
    assistant = ColabResearchAssistant(api_key, model)

    # Conduct research
    project_path = assistant.conduct_research(
        query=query,
        max_papers_per_source=max_papers,
        sources=["arxiv", "semantic_scholar", "pubmed"]
    )

    # Display summary
    assistant.display_project_summary()

    return assistant

def setup_research_environment():
    """Setup the research environment and return a configured assistant"""
    print("🔧 SETTING UP RESEARCH ENVIRONMENT")
    print("=" * 40)

    config = ColabConfig()
    api_key = config.setup_gemini_key()

    assistant = ColabResearchAssistant(api_key)

    print("Environment ready!")
    print("\n🚀 Quick start:")
    print("assistant.conduct_research('your research topic')")
    print("\n📚 Or use the quick function:")
    print("quick_research('your research topic')")

    return assistant

def quick_search(query: str, source: str = "arxiv", max_results: int = 5):
    """
    Quick search without full processing - just get paper titles and abstracts

    Args:
        query (str): Search query
        source (str): "arxiv", "semantic_scholar", or "pubmed"
        max_results (int): Number of results
    """
    config = ColabConfig()
    api_key = config.setup_gemini_key()

    assistant = ColabResearchAssistant(api_key)
    results = assistant.quick_search(query, source, max_results)

    print(f"🔍 Quick Search Results: '{query}' from {source}")
    print("=" * 50)

    for i, paper in enumerate(results, 1):
        print(f"{i}. **{paper['title']}**")
        print(f"   Authors: {paper['authors']}")
        print(f"   Source: {paper['source']}")
        print(f"   Abstract: {paper['abstract']}")
        print(f"   URL: {paper['url']}")
        print()

    return results

# ========================================
# EXAMPLE USAGE AND INSTRUCTIONS
# ========================================

def show_examples():
    """Show example usage patterns"""
    print("""
🔬 PERSONAL RESEARCH ASSISTANT - GEMINI VERSION
===============================================

🆓 **FREE Google Gemini API** - Get your key at: https://aistudio.google.com/app/apikey

## 🚀 QUICK START EXAMPLES

1. **INSTANT RESEARCH** (Easiest method):
   ```python
   assistant = quick_research("machine learning in healthcare", max_papers=15)
   ```

2. **SETUP ONCE, USE MULTIPLE TIMES**:
   ```python
   assistant = setup_research_environment()
   assistant.conduct_research("quantum computing algorithms", max_papers_per_source=12)
   ```

3. **QUICK PAPER SEARCH** (No AI processing):
   ```python
   results = quick_search("neural networks", source="arxiv", max_results=10)
   ```

4. **CUSTOM RESEARCH WITH MULTIPLE SOURCES**:
   ```python
   assistant.conduct_research(
       query="natural language processing transformers",
       max_papers_per_source=20,
       project_name="NLP_Transformers_2024",
       sources=["arxiv", "semantic_scholar", "pubmed"]
   )
   ```

## 📥 DOWNLOAD & MANAGE

```python
# Download current project
assistant.download_project()

# Download specific project
assistant.download_project("specific_project_name")

# List all projects
assistant.list_projects()
```

## 🆕 NEW FEATURES

✅ **Google Gemini AI** - Free API with generous limits
✅ **PubMed Integration** - Medical/biological research papers
✅ **Research Insights** - AI-generated trends and patterns analysis
✅ **Quick Search** - Fast paper discovery without full processing
✅ **Project Management** - List, download, and organize multiple projects

## 📊 SOURCES AVAILABLE

- **ArXiv** - Physics, Math, Computer Science, Biology
- **Semantic Scholar** - Multidisciplinary academic papers
- **PubMed** - Medical and life sciences literature

## 💡 PRO TIPS

• **Start small**: Begin with 8-12 papers per source for faster results
• **Use specific keywords**: "machine learning healthcare" vs "AI medicine"
• **Try different sources**: Each has different paper collections
• **Download results**: Save your work locally for future reference
• **Check insights**: New AI-generated research trends analysis

## 🔧 MODELS AVAILABLE

- **gemini-1.5-flash** (default) - Fast and efficient
- **gemini-1.5-pro** - More detailed analysis (slower)

## 📈 COST COMPARISON

**Google Gemini FREE**: 15 requests/minute, 1500 requests/day
**OpenAI GPT**: $0.002-0.06 per 1K tokens (paid only)

## 🚨 GETTING STARTED

```python
# Just run this and follow the prompts!
quick_research("your research topic here")
```

Happy researching! 🎓
    """)

def show_gemini_setup():
    """Show detailed Gemini API setup instructions"""
    print("""
🔑 GOOGLE GEMINI API SETUP GUIDE
================================

## Step 1: Get Your Free API Key
1. Go to: https://aistudio.google.com/app/apikey
2. Sign in with your Google account
3. Click "Create API Key"
4. Copy your API key (starts with 'AI...')

## Step 2: Run the Setup
```python
assistant = setup_research_environment()
```

## Step 3: Enter Your Key
- Paste your API key when prompted
- It will be securely stored for this session

## ✅ You're Ready!
```python
quick_research("your topic")
```

## 🆓 Free Limits (Very Generous!)
- 15 requests per minute
- 1,500 requests per day
- No credit card required

## 🔒 Privacy
- API keys are stored only in your Colab session
- Not saved permanently
- Google's standard privacy policies apply
    """)

# Auto-display examples when notebook loads
print("🎯 Personal Research Assistant - Gemini Version Ready!")
print("🆓 Powered by Google's FREE Gemini API")
print()
print("📚 Type: show_examples() to see usage examples")
print("🔑 Type: show_gemini_setup() for API setup help")
print("🚀 Type: quick_research('your topic') to start immediately")
print()
print("Get your FREE Gemini API key: https://aistudio.google.com/app/apikey")

🎯 Personal Research Assistant - Gemini Version Ready!
🆓 Powered by Google's FREE Gemini API

📚 Type: show_examples() to see usage examples
🔑 Type: show_gemini_setup() for API setup help
🚀 Type: quick_research('your topic') to start immediately

Get your FREE Gemini API key: https://aistudio.google.com/app/apikey


In [ ]:
# Run the research assistant code above first, then use:

# Method 1: Super quick research
assistant = quick_research("Microprocessor", max_papers=10)

# Method 2: Step-by-step setup
assistant = setup_research_environment()
assistant.conduct_research("RNN")

Google Gemini API Key Setup
Get your FREE API key at: https://aistudio.google.com/app/apikey

Gemini API key already configured!
STARTING RESEARCH PROJECT
Project: Microprocessor_20250824_095220
 Query: 'Microprocessor'
Max papers per source: 10
Sources: arxiv, semantic_scholar, pubmed
AI Model: Google Gemini

Project folder created: /content/research_projects/Microprocessor_20250824_095220

Searching ArXiv for: 'Microprocessor'
Found 10 papers from ArXiv
Searching Semantic Scholar for: 'Microprocessor'
Found 2 papers from Semantic Scholar
Searching PubMed for: 'Microprocessor'
Found 10 papers from PubMed

 PAPER COLLECTION SUMMARY
------------------------------
Total papers found: 22
Unique papers after deduplication: 22

Saved 22 papers to papers_data.json
GENERATING AI SUMMARIES WITH GEMINI
----------------------------------------
 Processing 1/22: Altermagnetic spintronics...
Summarizing: Altermagnetic spintronics...


Error summarizing paper: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: API key expired. Please renew the API key.
 Processing 2/22: Design and Simulation of 6T SRAM Array...
Summarizing: Design and Simulation of 6T SRAM Array...


Error summarizing paper: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: API key expired. Please renew the API key.
 Processing 3/22: Global Microprocessor Correctness in the Presence ...
Summarizing: Global Microprocessor Correctness in the Presence of Transie...
Error summarizing paper: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
 Processing 4/22: Detecting Hardware Trojans in Microprocessors via ...
Summarizing: Detecting Hardware Trojans in Microprocessors via Hardware E...


Error summarizing paper: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: API key expired. Please renew the API key.
 Processing 5/22: Role of Uncertainty in Model Development and Contr...
Summarizing: Role of Uncertainty in Model Development and Control Design ...


Error summarizing paper: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: API key expired. Please renew the API key.
 Processing 6/22: Control Architecture and Design for a Multi-roboti...
Summarizing: Control Architecture and Design for a Multi-robotic Visual S...
Error summarizing paper: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
 Processing 7/22: Barium Calcium Zirconium Titanate Thin Film-Based ...
Summarizing: Barium Calcium Zirconium Titanate Thin Film-Based Capacitive...


...

Error summarizing paper: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: API key expired. Please renew the API key.
 Processing 13/23: Dual-Path RNN: Efficient Long Sequence Modeling fo...
Summarizing: Dual-Path RNN: Efficient Long Sequence Modeling for Time-Dom...
Error summarizing paper: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
 Processing 14/23: Transformer Transducer: A Streamable Speech Recogn...
Summarizing: Transformer Transducer: A Streamable Speech Recognition Mode...


Error summarizing paper: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: API key expired. Please renew the API key.
 Processing 15/23: CNN-RNN: A Unified Framework for Multi-label Image...
Summarizing: CNN-RNN: A Unified Framework for Multi-label Image Classific...
Error summarizing paper: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
 Processing 16/23: Development of student intent-based educational ch...
Summarizing: Development of student intent-based educational chatbot syst...


Error summarizing paper: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: API key expired. Please renew the API key.
 Processing 17/23: Performance analysis of neural network architectur...
Summarizing: Performance analysis of neural network architectures for tim...
Error summarizing paper: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
 Processing 18/23: Predictive efficacy of machine-learning algorithms...
Summarizing: Predictive efficacy of machine-learning algorithms on intrah...


Error summarizing paper: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: API key expired. Please renew the API key.
 Processing 19/23: CWBLS network and its application in portable spec...
Summarizing: CWBLS network and its application in portable spectral measu...
Error summarizing paper: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
 Processing 20/23: A Joint Multimodal User Authentication-based Priva...
Summarizing: A Joint Multimodal User Authentication-based Privacy Preserv...


Error summarizing paper: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: API key expired. Please renew the API key.
 Processing 21/23: Emotion recognition in EEG Signals: Deep and machi...
Summarizing: Emotion recognition in EEG Signals: Deep and machine learnin...
Error summarizing paper: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
 Processing 22/23: Mechanobiology-guided machine learning models for ...
Summarizing: Mechanobiology-guided machine learning models for predicting...
